In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [24]:
llm_model = "gpt-3.5-turbo"
# llm_model = "gpt-3.5-turbo-0301"

In [25]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [26]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [10]:
docs = loader.load()
print(docs[0])

page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries." metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}


In [12]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [14]:
embeded = embeddings.embed_query("Hi, This is Shubho Chowdhury")
print(len(embeded), embeded[:5])

1536 [0.006724867504090071, 0.00954195111989975, -0.01940508931875229, 0.0055873277597129345, -0.028826596215367317]


In [16]:
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings
)

In [39]:
query = "is there any cloths for winter?"

In [40]:
docs = db.similarity_search(query)

In [41]:
len(docs)

4

In [42]:
docs[0]

Document(page_content=': 760\nname: Winterproof Snow Trousers\ndescription: These men’s waterproof snow pants are designed for natural movement and maximum warmth and waterproof breathability. The breathable, waterproof TEK2® 100% nylon fabric is taped at the seams and insulated with our ThermaTEK polyester insulation. Snap closure and waistband adjustments provide a customizable fit over any base layer, and snow gaiter hems keep out deep snow and allow for easy on/off over boots. The lining content is 100% Nylon taffeta and the pants are machine washable and dryable. \n\nSize & Fit: Relaxed Fit - our loosest cut, with the most room through the seat and knee. \n\nImported. \n\nIf you have any questions, please reach out to our customer service team.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 760})

In [43]:
retriever = db.as_retriever()

In [45]:
llm = ChatOpenAI(temperature=0.0)

In [46]:
q_docs = " ".join(i.page_content for i in docs)

In [47]:
q_docs

": 760\nname: Winterproof Snow Trousers\ndescription: These men’s waterproof snow pants are designed for natural movement and maximum warmth and waterproof breathability. The breathable, waterproof TEK2® 100% nylon fabric is taped at the seams and insulated with our ThermaTEK polyester insulation. Snap closure and waistband adjustments provide a customizable fit over any base layer, and snow gaiter hems keep out deep snow and allow for easy on/off over boots. The lining content is 100% Nylon taffeta and the pants are machine washable and dryable. \n\nSize & Fit: Relaxed Fit - our loosest cut, with the most room through the seat and knee. \n\nImported. \n\nIf you have any questions, please reach out to our customer service team. : 414\nname: Cozy Comfort Everyday Underwear, Pants Pattern\ndescription: Stay cozy with our heat-generating underwear, designed to keep you up to 7 degrees warmer. Crafted with 48% acrylic, 28% Tencel, 19% polyester, and 5% spandex, these underwear are trim-fit

In [50]:
response = llm.call_as_llm(f"Question: Please analyze the given content and list all shirts for cold temperature in a table in a markdown and summarize each one\n\nContent:{q_docs}")

In [51]:
display(Markdown(response))

| Shirt Name | Description |
|------------|-------------|
| Winterproof Snow Trousers | These men’s waterproof snow pants are designed for natural movement and maximum warmth and waterproof breathability. The breathable, waterproof TEK2® 100% nylon fabric is taped at the seams and insulated with our ThermaTEK polyester insulation. Snap closure and waistband adjustments provide a customizable fit over any base layer, and snow gaiter hems keep out deep snow and allow for easy on/off over boots. The lining content is 100% Nylon taffeta and the pants are machine washable and dryable. |
| Cozy Comfort Everyday Underwear, Pants Pattern | Stay cozy with our heat-generating underwear, designed to keep you up to 7 degrees warmer. Crafted with 48% acrylic, 28% Tencel, 19% polyester, and 5% spandex, these underwear are trim-fitting and ultra-soft, perfect for wear under clothing or on their own. The advanced fabric traps body moisture, transforming it into heat, even while sitting still. Machine wash and dry for easy care. Perfect for casual warmth and everyday wear while running errands, walking the dog or snow blowing. Imported. |
| Dual-Layer Undershorts | Made to put cold weather in its place, our men's double-layer thermal underwear is built to last for years of cozy outdoor activity. Size & Fit: Snug fit assures comfortable fit beneath outer layers. Fabric & Care: Inside is soft, itch-free cotton. Outside is a durable, warm blend of cotton and merino wool. Machine wash and dry. Construction: Supersoft, warm two-layer construction won’t bind. Nonchafing flatlock seams for superior comfort. Rib-knit cuffs with Lycra elastane hold their shape. Elastic waist with fly front. Imported. |
| Ladies' Luxury Snow Coat | This insulated soft-shell jacket is extra rugged, with just the right amount of stretch so you can move smoothly down the mountain. It provides light activity protection up to 30° and moderate activity protection up to -20°. The slightly fitted design is best when worn with a midweight layer and falls at the hip. It is constructed of a durable and highly weather-resistant stretch soft shell blend of 88% nylon and 12% elastane, with ultralight, ultrawarm PrimaLoft insulation to keep you comfortable and dry on the slopes. Additional features include a longer length and higher collar for extra protection, a helmet-compatible, fully adjustable hood, pit zip venting, an interior media pocket and goggle pocket, and a drawcord at the hem. This jacket is machine washable and dryable and is imported. If you have any questions, please contact us. |

Summary:
1. Winterproof Snow Trousers: Waterproof snow pants designed for natural movement and maximum warmth. Made of breathable, waterproof TEK2® nylon fabric with ThermaTEK polyester insulation. Customizable fit with snap closure and waistband adjustments. Snow gaiter hems keep out deep snow. Machine washable and dryable.
2. Cozy Comfort Everyday Underwear, Pants Pattern: Heat-generating underwear designed to keep you up to 7 degrees warmer. Made of acrylic, Tencel, polyester, and spandex. Trim-fitting and ultra-soft. Traps body moisture and transforms it into heat. Machine wash and dry. Perfect for casual warmth and everyday wear.
3. Dual-Layer Undershorts: Double-layer thermal underwear for cozy outdoor activity. Snug fit beneath outer layers. Soft, itch-free cotton inside and durable, warm blend of cotton and merino wool outside. Nonchafing flatlock seams. Rib-knit cuffs with Lycra elastane. Elastic waist with fly front.
4. Ladies' Luxury Snow Coat: Insulated soft-shell jacket with stretch for smooth movement. Provides light activity protection up to 30° and moderate activity protection up to -20°. Slightly fitted design for wearing with a midweight layer. Made of durable and weather-resistant stretch soft shell blend. Ultralight, ultrawarm PrimaLoft insulation. Additional features include longer length, higher collar, helmet-compatible hood, pit zip venting, interior media pocket, goggle pocket, and drawcord at the hem. Machine washable and dryable.

### RetrievalQA

In [63]:
qa_stuff = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type='map_reduce',
    retriever=retriever,
    verbose=True
)

In [64]:
query = "Please analyze the given content and list all shirts for cold temperature in a table in a markdown and summarize each one"

In [ ]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...


In [ ]:
display(Markdown(response))

### Alternative

In [60]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

In [61]:
response = index.query(query, llm=llm)

In [62]:
display(Markdown(response))

| Shirt ID | Shirt Name                           | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                